#Introduction

This Colab notebook aims to explore three popular techniques for information retrieval: BM25, BOW, and TF-IDF. In this notebook, I will be utilizing the Pyserini library to implement the BM25 algorithm.

Firstly, I will provide a simple implementation of these three techniques and assess their performance on the MsMarco subset: TRECDL-2020. By the end of this notebook, a performance comparison between the algorithms will be presented.

Usually, when evaluating an information retrieval system, it is essential to use a standardized format to compare the results obtained by different algorithms. In this context, the Text Retrieval Conference (TREC) format is widely used. 
<br>
<br>
To do this, we generally use three files: (1) topics (user queries), (2) qrels (query relevance), indicating the relevant documents for each query, and (3) the run file containing the ranking information.

# BM25 - Pyserini

Install the Pyserini library and faiss-cpu (used by Pyserini)

In [1]:
!pip install pyserini -q
!pip install faiss-cpu==1.7.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 955.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The first step is to get the topics dictionary containing the query IDs and text

In [2]:
from pyserini.search import get_topics

topics = get_topics('dl20')

In [3]:
# topics HEAD
dict(list(topics.items())[:10])

{735922: {'title': 'what is crimp oil'},
 23849: {'title': 'are naturalization records public information'},
 514096: {'title': 'the after hours clinic'},
 156498: {'title': 'do google docs auto save'},
 258062: {'title': 'how long does it take to remove wisdom tooth'},
 703782: {'title': 'what is a torn disc'},
 1071750: {'title': 'why is pete rose banned from hall of fame'},
 436707: {'title': 'largest known insects'},
 1117817: {'title': 'what does unauthorized act in writing mean'},
 1135268: {'title': 'antibiotics for what kind of infection'}}

The second step is to download a pre-built index version of the MsMarco-Passage dataset that contains the documents we want to rank. This can be accomplished by creating a LuceneSearcher object.

In [4]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher.from_prebuilt_index('msmarco-passage')

index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [01:20, 27.7MB/s]                            


Now, performing the information retrieval, we need to format and output the results in a way that can be evaluated using standard metrics. To achieve this, we create a RUN file in the TREC format, which specifies the ranking of documents for each query in the evaluation dataset. This allows us to compare the performance of different techniques using established evaluation measures.
<br>
<br>
TREC format:
<br>
`topic_id Q0 doc_id rank score label `

In [5]:
from tqdm.notebook import tqdm

def get_run(path: str, topics: dict, top_k: int, searcher):
    with open(path, 'w') as fout:
        for id in tqdm(topics, desc="Running queries"):
            query = topics[id]['title']
            hits = searcher.search(query, top_k)
            for idx, hit in enumerate(hits):
              fout.write(f"{id}\tQ0\t{hit.docid}\t{idx+1}\t{hit.score}\tBM25\n")

In [6]:
get_run('run-msmarco-bm25.tsv', topics, 1000, searcher)

Running queries:   0%|          | 0/200 [00:00<?, ?it/s]

# Corpus

Downloads the MSMarco corpus

In [7]:
# Downloads the MsMarco dataset
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

--2023-03-07 21:06:43--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  6.79MB/s    in 2m 58s  

2023-03-07 21:09:42 (5.66 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


In [8]:
# Number of documents (MsMarco)
!wc -l /content/collections/msmarco-passage/collection.tsv

8841823 /content/collections/msmarco-passage/collection.tsv


Uses nltk to get a list of stopwords to filter during the indexing

In [9]:
import nltk
nltk.download('stopwords') # downloads the stopwords dataset
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from collections import defaultdict
from tqdm.notebook import tqdm
from spacy.lang.en import English


class Dataloader():
  def __init__(self, corpus_path: str):
    self.corpus = corpus_path
    self.stop_words = set(stopwords.words('english'))
    self.inverted_index = defaultdict(lambda: defaultdict(int))
    self.nlp = English()
    self.load_corpus(corpus_path)

  def filter_stop_words(self, words: list):
    return [word for word in words if word not in self.stop_words]

  def tokenize(self, document: str):
    doc = self.nlp(document)
    return [str(token.text).lower() for token in doc if not token.is_punct]

  def load_corpus(self, corpus: str):
    '''
    Each line of the corpus file represents one document.
    '''
    count = 0
    with open(corpus, "r") as file:
      for line in tqdm(file, desc="Loading corpus", total=8841823):
        count += 1
        id, content = line.strip().split("\t")
        tokens = self.tokenize(content)
        filtered_tokens = self.filter_stop_words(tokens)
        for token in filtered_tokens:
            self.inverted_index[token][str(id)] += 1
    self.corpus_length = count

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [11]:
dataset = Dataloader("/content/collections/msmarco-passage/collection.tsv")

Loading corpus:   0%|          | 0/8841823 [00:00<?, ?it/s]

In [12]:
# RAM usage so far
!free -h

              total        used        free      shared  buff/cache   available
Mem:           25Gi        15Gi       388Mi       1.0Mi       9.5Gi       9.5Gi
Swap:            0B          0B          0B


# BoW
The Bag of Words (BoW) technique is a common approach used in search engines to represent documents as vectors of word frequencies. In this technique, a document is first pre-processed to remove stop words, punctuation, and other noise. When a query is entered into the search engine, it is also converted into a vector using the same BoW technique. The search engine then computes the similarity between the query vector and each document vector, returning the documents that are most similar to the query. The BoW technique is simple yet effective, but it has limitations, such as ignoring the order of words and not considering the meaning of words in context. 

The implementation below uses the inverted index to speed up the search.

In [13]:
class BoW():
    def __init__(self, dataset, topics, run_path):
        self.dataset = dataset
        self.nlp = English()
        self.stop_words = set(stopwords.words('english'))
        self.get_run(topics, run_path)

    def get_score(self, query):
        """
        Get the scores for a given query.
        Args:
            query: the query being searched
        """
        query = self.nlp(query)
        tokens = set([str(token.text).lower() for token in query if not token.is_punct])
        # Remove stopwords
        filtered_tokens = [token for token in tokens if token not in self.stop_words]
        scores = defaultdict(int)
        for token in filtered_tokens:
            # A document score is the sum of all token frequencies that
            # occur in the query
            for doc in self.dataset.inverted_index[token].keys():
                scores[doc] += self.dataset.inverted_index[token][doc] 
        return scores

    def get_run(self, topics, run_path):
        """
        Creates a TREC run.
        Args:
            topics: file containing all queries
            run_path: path to save the RUN
        """
        with open(run_path, "w") as fout:
            for qid, query in tqdm(topics.items(), desc="Getting scores"):
                scores = self.get_score(query["title"])
                sorted_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)[:1000]
                count = 0
                for docid, score in sorted_docs:
                    count += 1
                    fout.write(f"{qid}\tQ0\t{docid}\t{count}\t{score}\tBoW\n")


In [14]:
bow = BoW(dataset, topics, "BoW_run.tsv")

Getting scores:   0%|          | 0/200 [00:00<?, ?it/s]

In [15]:
# RAM Usage
!free -h

              total        used        free      shared  buff/cache   available
Mem:           25Gi        15Gi       420Mi       1.0Mi       9.4Gi       9.5Gi
Swap:            0B          0B          0B


# TF-IDF

The idea behind TF-IDF is to give more weight to terms that are rare in a collection of documents and more informative about the content. The technique first calculates the term frequency (TF) of each term in a document, which represents the number of times the term appears in the document. The inverse document frequency (IDF) is then computed for each term, which measures how rare the term is in the entire collection of documents. The final TF-IDF weight for a term is the product of its TF and IDF values. This way, terms that are common in a document but also common across the collection of documents will have lower weights than terms that are rare in the collection but appear frequently in a particular document.

In [16]:
import math
import time

class TFIDF:
    def __init__(self, dataset, topics, run_path):
        self.dataset = dataset
        self.nlp = English()
        self.stop_words = set(stopwords.words('english'))
        self.dataset_length = dataset.corpus_length
        self.scores = self.get_run(topics, run_path)

    def get_score(self, query: str):
        """
        Computes the score for a given query
        Args:
            query: the query being searched
        Returns:
            a dict containing all document scores
        """
        query = self.nlp(query)
        tokens = set([str(token.text).lower() for token in query if not token.is_punct])
        # Removes the stopwords
        filtered_tokens = [token for token in tokens if token not in self.stop_words]
        scores = defaultdict(int)
        for token in filtered_tokens:
            # Number of documents with this token
            num_docs = len(self.dataset.inverted_index[token])
            idf = math.log(self.dataset_length / (1 + num_docs))
            for doc in self.dataset.inverted_index[token].keys():
                tf = self.dataset.inverted_index[token][doc]
                scores[doc] += tf*idf
        return scores

    def get_run(self, topics, run_path):
        """
        Creates a TREC run.
        Args:
            topics: file containing all queries
            run_path: path to save the RUN
        """
        with open(run_path, "w") as fout:
            for qid, query in tqdm(topics.items(), desc="Getting scores"):
                scores = self.get_score(query["title"])
                sorted_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)[:1000]
                count = 0
                for docid, score in sorted_docs:
                    count += 1
                    fout.write(f"{qid}\tQ0\t{docid}\t{count}\t{score}\tTF-IDF\n")


In [17]:
tf_idf = TFIDF(dataset, topics, "tfidf_run.tsv")

Getting scores:   0%|          | 0/200 [00:00<?, ?it/s]

In [18]:
# RAM Usage
!free -h

              total        used        free      shared  buff/cache   available
Mem:           25Gi        15Gi       426Mi       1.0Mi       9.4Gi       9.4Gi
Swap:            0B          0B          0B


# Evaluation
Finally, to assess the performance of BM25, we calculate the normalized discounted cumulative gain (nDCG) at rank 10 for the queries in our evaluation dataset. A value of 1 indicates perfect ranking, while lower values indicate poorer performance. By evaluating our system using the nDCG@10 metric, we can compare its performance to other systems and determine whether it is effective for the given task.

In [19]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mmap -l 2 dl20-passage run-msmarco-bm25.tsv

2023-03-07 21:54:29.593457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 21:54:29.593670: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 21:54:29.593697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 4.19MB/s]                
Running command: ['java', '

## TF-IDF

In [20]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mmap -l 2 dl20-passage tfidf_run.tsv

2023-03-07 21:54:41.570157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 21:54:41.570281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 21:54:41.570303: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running

## BoW

In [21]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mmap -l 2 dl20-passage BoW_run.tsv

2023-03-07 21:54:52.997346: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 21:54:52.997469: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-07 21:54:52.997488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running

In [22]:
# Final RAM usage
!free -h

              total        used        free      shared  buff/cache   available
Mem:           25Gi        15Gi       3.3Gi       1.0Mi       6.5Gi       9.4Gi
Swap:            0B          0B          0B


# Final Results
In conclusion, the BM25 ranking function has been shown to outperform both TF-IDF and Bag-of-Words (BoW) approaches in terms of retrieval effectiveness for the MsMarco subset (TRECDL 2020). BM25 is a more sophisticated ranking function that considers document length and term frequency normalization, which are particularly important for longer documents such as web pages. TF-IDF outperformed Bag-of-Words (BoW) approach since it takes into account not only the frequency of each term in a document but also the importance of the term in the entire collection of documents.

| Model |  MAP   | nDCG@10 |
|-------|------- |---------|
| BM25  | 0.2857 |  0.4803 |
| BoW   | 0.0348 |  0.0639 |
| TF-IDF| 0.0702 |  0.1239 | 
